In [ ]:
#| default_exp core

# dialoghelper

In [ ]:
#| export
import inspect, json, importlib, linecache
from typing import Dict
from tempfile import TemporaryDirectory
from ipykernel_helper import *

from fastcore.utils import *
from fastcore.meta import delegates
from ghapi.all import *
from fastlite import *
from fastcore.xtras import asdict

In [ ]:
from solveit.db import *

In [ ]:
#| export
_all_ = ["asdict"]

In [ ]:
#| export
def get_db(ns:dict=None):
    app_path = Path('/app') if Path('/.dockerenv').exists() else Path('.')
    if os.environ.get('IN_SOLVEIT', False): dataparent,nm = app_path, 'data.db'
    else: dataparent,nm = Path('..'),'dev_data.db'
    db = database(dataparent/'data'/nm)
    dcs = [o for o in all_dcs(db) if o.__name__[0]!='_']
    if ns:
        for o in dcs: ns[o.__name__]=o
    return db

In [ ]:
# db.conn.filename

In [ ]:
db = get_db(globals())
dlg = db.t.dialog.fetchone()
dlg

Dialog(id=1, name='test dialog', mode=2)

In [ ]:
#| export
def find_var(var:str):
    "Search for var in all frames of the call stack"
    frame = inspect.currentframe()
    while frame:
        dv = frame.f_globals.get(var, frame.f_locals.get(var, None))
        if dv: return dv
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

In [ ]:
a = 1
find_var('a')

1

In [ ]:
#| export
def find_dialog_id():
    "Get the dialog id by searching the call stack for __dialog_id."
    return find_var('__dialog_id')

In [ ]:
__dialog_id = dlg.id

In [ ]:
find_dialog_id()

1

In [ ]:
#| export
def find_msgs(
    pattern:str='', # Optional text to search for
    msg_type:str=None, # optional limit by message type ('code', 'note', or 'prompt')
    limit:int=None, # Optionally limit number of returned items
    include_output:bool=True # Include output in returned dict?
):
    "Find messages in current specific dialog that contain the given information. To refer to a message found later, use its `sid` field (which is the pk)."
    did = find_dialog_id()
    db = get_db()
    res = db.t.message('did=? AND content LIKE ? ORDER BY mid', [did, f'%{pattern}%'], limit=limit)
    res = [asdict(o) for o in res if not msg_type or (msg_type==o.msg_type)]
    if not include_output:
        for o in res: o.pop('output', None)
    return res

In [ ]:
found = find_msgs('hello')
found[0]['content']

'hello world'

In [ ]:
find_msgs(msg_type='prompt', include_output=False)

[{'sid': '_W_YSjhjCR5ehQup9F74xEQ',
  'mid': 'msg-a0',
  'content': 'Concisely, what is *one* simple way to reverse a list?',
  'input_tokens': 24,
  'output_tokens': 54,
  'msg_type': 'prompt',
  'time_run': '',
  'is_exported': 0,
  'skipped': 0,
  'did': 1,
  'i_collapsed': 0,
  'o_collapsed': 0,
  'header_collapsed': 0,
  'pinned': 0}]

In [ ]:
#| export
def find_msg_id():
    "Get the message id by searching the call stack for __dialog_id."
    return find_var('__msg_id')

In [ ]:
__msg_id = found[0]['sid']

In [ ]:
find_msg_id()

'_Q7ejppqNSgOYDXtx2PVkEw'

In [ ]:
#| export
def read_msg_ids():
    "Get all ids in current dialog."
    did = find_dialog_id()
    db = get_db()
    return [o.sid for o in db.t.message('did=?', [did], select='sid', order_by='mid')]

In [ ]:
#| export
def msg_idx():
    "Get relative index of current message in dialog."
    ids = read_msg_ids()
    return ids,ids.index(find_msg_id())

In [ ]:
ids,idx = msg_idx()
idx

1

In [ ]:
#| export
def read_msg(n:int=-1,     # Message index (if relative, +ve is downwards)
             relative:bool=True  # Is `n` relative to current message (True) or absolute (False)?
    ):
    "Get the message indexed in the current dialog."
    ids,idx = msg_idx()
    if relative:
        idx = idx+n
        if not 0<=idx<len(ids): return None
    else: idx = n
    db = get_db()
    return db.t.message.fetchone('sid=?', [ids[idx]])

In [ ]:
# Previous message relative to current
read_msg(-1)

Message(sid='_W_YSjhjCR5ehQup9F74xEQ', mid='msg-a0', content='Concisely, what is *one* simple way to reverse a list?', output='You can reverse a list using the slice notation with a step of -1:\n\n```python\nmy_list[::-1]\n```\n\nWould you like me to explain how this works?', input_tokens=24, output_tokens=54, msg_type='prompt', time_run='', is_exported=0, skipped=0, did=1, i_collapsed=0, o_collapsed=0, header_collapsed=0, pinned=0)

In [ ]:
# Last message in dialog
read_msg(-1, relative=False)

Message(sid='_7BuMofkeTUyf9Jt4iUY_hQ', mid='msg-a3', content='1+1 / 12', output='[{"output_type": "execute_result", "metadata": {}, "data": {"text/plain": ["1.0833333333333333"]}, "execution_count": 22}]', input_tokens=9, output_tokens=30, msg_type='code', time_run='', is_exported=0, skipped=0, did=1, i_collapsed=0, o_collapsed=0, header_collapsed=0, pinned=0)

In [ ]:
#| export
def _msg(
    time_run: str | None = '', # When was message executed
    is_exported: int | None = 0, # Export message to a module?
    skipped: int | None = 0, # Hide message from prompt?
    i_collapsed: int | None = 0, # Collapse input?
    o_collapsed: int | None = 0, # Collapse output?
    header_collapsed: int | None = 0, # Collapse heading section?
    pinned: int | None = 0 # Pin to context?
): ...

In [ ]:
#| export
@delegates(_msg)
def add_msg(
    content:str, # Content of the message (i.e the message prompt, code, or note text)
    msg_type: str='note', # Message type, can be 'code', 'note', or 'prompt'
    output:str='', # For prompts/code, initial output
    placement:str='add_after', # Can be 'add_after', 'add_before', 'update', 'at_start', 'at_end'
    sid:str=None, # sid (stable id -- pk) of message that placement is relative to (if None, uses current message)
    **kwargs
):
    "Add/update a message to the queue to show after code execution completes. Be sure to pass a `sid` (stable id) not a `mid` (which is used only for sorting, and can change)."
    assert msg_type in ('note', 'code', 'prompt'), "msg_type must be 'code', 'note', or 'prompt'."
    assert msg_type not in ('note') or not output, "'note' messages cannot have an output."
    run_cmd('add_msg', content=content, msg_type=msg_type, output=output, placement=placement, sid=sid, **kwargs)

In [ ]:
#| export
def _umsg(
    content:str|None = None, # Content of the message (i.e the message prompt, code, or note text)
    msg_type: str|None = None, # Message type, can be 'code', 'note', or 'prompt'
    output:str|None = None, # For prompts/code, the output
    time_run: str | None = None, # When was message executed
    is_exported: int | None = None, # Export message to a module?
    skipped: int | None = None, # Hide message from prompt?
    i_collapsed: int | None = None, # Collapse input?
    o_collapsed: int | None = None, # Collapse output?
    header_collapsed: int | None = None, # Collapse heading section?
    pinned: int | None = None # Pin to context?
): ...

In [ ]:
#| export
@delegates(_umsg)
def update_msg(
    msg:Optional[Dict]=None, # Dictionary of field keys/values to update
    sid:str=None, # sid (stable id -- pk) of message to update (if None, uses current message)
    **kwargs):
    """Update an existing message. Provide either `msg` OR field key/values to update.
    Use `content` param to update contents. Be sure to pass a `sid` (stable id -- the pk) not a `mid`
    (which is used only for sorting, and can change).
    Only include parameters to update--missing ones will be left unchanged."""
    kw = (msg or {}) | kwargs
    if not sid: raise TypeError("update_msg needs either a dict message or `sid=...`")
    kw.pop('did', None)
    run_cmd('add_msg', placement='update', sid=sid, **kw)

In [ ]:
#| export
def add_html(
    html:str, # HTML to add to the DOM
):
    "Dynamically add HTML to the current web page. Supports HTMX attrs too."
    run_cmd('add_ft', html=html)

In [ ]:
#| export
def load_gist(gist_id:str):
    "Retrieve a gist"
    api = GhApi()
    if '/' in gist_id: *_,user,gist_id = gist_id.split('/')
    else: user = None
    return api.gists.get(gist_id, user=user)

In [ ]:
gistid = 'jph00/e7cfd4ded593e8ef6217e78a0131960c'
gist = load_gist(gistid)
gist.html_url

'https://gist.github.com/jph00/e7cfd4ded593e8ef6217e78a0131960c'

In [ ]:
#| export
def gist_file(gist_id:str):
    "Get the first file from a gist"
    gist = load_gist(gist_id)
    return first(gist.files.values())

In [ ]:
gfile = gist_file(gistid)
print(gfile.content)

testfoo='testbar'


In [ ]:
#| export
def import_string(
    code:str, # Code to import as a module
    name:str  # Name of module to create
):
    with TemporaryDirectory() as tmpdir:
        path = Path(tmpdir) / f"{name}.py"
        path.write_text(code)
        # linecache.cache storage allows inspect.getsource() after tmpdir lifetime ends
        linecache.cache[str(path)] = (len(code), None, code.splitlines(keepends=True), str(path))
        spec = importlib.util.spec_from_file_location(name, path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[name] = module
        spec.loader.exec_module(module)
        return module

In [ ]:
#| export
def import_gist(
    gist_id:str, # user/id or just id of gist to import as a module
    mod_name:str=None, # module name to create (taken from gist filename if not passed)
    add_global:bool=True # add module to caller's globals?
):
    "Import gist directly from string without saving to disk"
    fil = gist_file(gist_id)
    mod_name = mod_name or Path(fil['filename']).stem
    module = import_string(fil['content'], mod_name)
    if add_global: inspect.currentframe().f_back.f_globals[mod_name] = module
    return module

In [ ]:
import_gist(gistid)
importtest.testfoo

'testbar'

## export -

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()